![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-ocr-workshop/blob/TrainingNotebooks/tutorials/Certification_Trainings/2.1.Pdf_processing.ipynb)

## Blogposts and videos

- [Text Detection in Spark OCR](https://medium.com/spark-nlp/text-detection-in-spark-ocr-dcd8002bdc97)

- [Table Detection & Extraction in Spark OCR](https://medium.com/spark-nlp/table-detection-extraction-in-spark-ocr-50765c6cedc9)

- [Extract Tabular Data from PDF in Spark OCR](https://medium.com/spark-nlp/extract-tabular-data-from-pdf-in-spark-ocr-b02136bc0fcb)

- [Signature Detection in Spark OCR](https://medium.com/spark-nlp/signature-detection-in-spark-ocr-32f9e6f91e3c)

- [GPU image pre-processing in Spark OCR](https://medium.com/spark-nlp/gpu-image-pre-processing-in-spark-ocr-3-1-0-6fc27560a9bb)

- [How to Setup Spark OCR on UBUNTU - Video](https://www.youtube.com/watch?v=cmt4WIcL0nI)


**More examples here**

https://github.com/JohnSnowLabs/spark-ocr-workshop

For get the trial license please go to:

https://www.johnsnowlabs.com/install/

In [ ]:
## Setup

In [1]:
import os
import pkg_resources
from pyspark.ml import PipelineModel
import sparkocr
from sparkocr.transformers import *
from sparkocr.utils import *

import warnings
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'


spark = start_spark()
print("Spark OCR Version :", sparkocr.version())

## Read pdfs to the dataframe and display

In [ ]:
pdf_path = pkg_resources.resource_filename('sparkocr', 'resources/ocr/pdfs/*.pdf')
pdf_example_df = spark.read.format("binaryFile").load(pdf_path).cache()

display_pdf(pdf_example_df)

## Define pipeline for extract text from the searchable and ocr text from the scaned pdf

In [7]:
def pipeline():
    
    # If text PDF extract text
    pdf_to_text = PdfToText() \
        .setInputCol("content") \
        .setOutputCol("text") \
        .setSplitPage(True) \
        .setExtractCoordinates(True) \
        .setStoreSplittedPdf(True)
    
    # If image pdf, extract image
    pdf_to_image = PdfToImage() \
        .setInputCol("content") \
        .setOutputCol("image") \
        .setKeepInput(True)
    
    # Run OCR
    ocr = ImageToText() \
        .setInputCol("image") \
        .setOutputCol("text") \
        .setConfidenceThreshold(60)

    pipeline = PipelineModel(stages=[
        pdf_to_text,
        pdf_to_image,
        ocr
    ])
    
    return pipeline

## Run pipeline and show results

In [51]:
result = pipeline().transform(pdf_example_df).cache()
result.show()

+--------------------+-------------------+------+--------------------+--------------------+-----------------+------------------+--------------------+--------------------+-----------+-------+-----------+----------------+---------+
|                path|   modificationTime|length|                text|           positions| height_dimension|   width_dimension|             content|               image|total_pages|pagenum|documentnum|      confidence|exception|
+--------------------+-------------------+------+--------------------+--------------------+-----------------+------------------+--------------------+--------------------+-----------+-------+-----------+----------------+---------+
|file:/Users/nmeln...|2022-07-14 15:38:51|693743|Patient Nam\nFina...|[{[{Patient Nam\n...|1587.780029296875|1205.8299560546875|[25 50 44 46 2D 3...|{file:/Users/nmel...|          1|      0|          0|81.2276874118381|     null|
|file:/Users/nmeln...|2022-07-14 15:38:51|693743|Patient Name\nFin...|[{[{Patien

## Display text using pandas dataframe

In [53]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

result.select("text").toPandas().style.set_properties(**{'white-space': 'pre-wrap', 'text-align': 'left'})